# Prototype Loteca to BetExplorer matches

## Test pandas speed

We are gonna use pandas a lot here. So, let's test
base speed of retrieval of an item vs a dict.

Let's test a namedtuple also.

In [8]:
from collections import namedtuple
import pandas as pd

Obj = namedtuple('Obj', 'a, b, c')
a = pd.Series([1, 2, 3], index=['a', 'b', 'c'])
b = {'a': 1, 'b': 2, 'c': 3}
c = Obj(1, 2, 3)

In [9]:
%timeit a.loc['a']
%timeit b['a']
%timeit c.a

37.9 µs ± 384 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)
59.8 ns ± 0.258 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)
79.6 ns ± 0.542 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


pandas is about 1000 times slower than the python alternatives.

This should be taken in consideration.

## Misc

In [1]:
import sys
sys.path.append('..')

In [2]:
from src.util import load_pickle

df = load_pickle('../data/process/loteca_matches.pkl')

In [3]:
df.head()

,roundno,gameno,date,team_h,goals_h,team_a,goals_a,happened
5110,366,1,2009-06-07,CRUZEIRO/MG,1,INTERNACIONAL/RS,1,True
5111,366,2,2009-06-07,ATLÉTICO/PR,0,ATLÉTICO/MG,4,True
5112,366,3,2009-06-07,AVAÍ/SC,0,SÃO PAULO/SP,0,True
5113,366,4,2009-06-06,AMÉRICA/RN,2,BRASILIENSE/DF,1,True
5114,366,5,2009-06-06,PONTE PRETA/SP,5,PORTUGUESA DESPORTOS/SP,2,True


In [17]:
import sqlite3

import pandas as pd

conn = sqlite3.connect('../data/db.sqlite3')
q = "SELECT * FROM betexp_matches"
df = pd.read_sql_query(q, conn)
conn.close()

In [18]:
df.head()

,id,url,league_category,league_name,league_year,team_h,team_a,date,score,scoremod
0,bqHgC1sl,http://www.betexplorer.com/soccer/world/arab-c...,world,Arab Champions League,2008/2009,Esperance Tunis,Wydad,21.05.2009,1:1,
1,GKRlDsSs,http://www.betexplorer.com/soccer/world/arab-c...,world,Arab Champions League,2008/2009,Wydad,Esperance Tunis,09.05.2009,0:1,
2,pxyJI34K,http://www.betexplorer.com/soccer/world/arab-c...,world,Arab Champions League,2008/2009,Esperance Tunis,ES Setif,26.04.2009,2:0,
3,EBuNHNJQ,http://www.betexplorer.com/soccer/world/arab-c...,world,Arab Champions League,2008/2009,Wydad,Sfaxien,25.04.2009,2:0,
4,6oxFJqkE,http://www.betexplorer.com/soccer/world/arab-c...,world,Arab Champions League,2008/2009,Sfaxien,Wydad,12.04.2009,1:1,


In [74]:
df.shape

(299367, 10)

In [71]:
from src.data.interim.teams import betexplorer

# remove duplicates
df = df.drop_duplicates(subset='id')

# process necessary columns
def get_date(s):
    d, m, y = [int(v) for v in s.split('.')]
    return date(y, m, d)

def get_score(s):
    if s == '':
        return None
    
    s = s.strip()
    return [int(v) for v in s.split(':')]

def process_name(s):
    name, _, women_flag, under, _ = betexplorer.parse_string(s)
    fname = betexplorer.format_name(name)
    return (fname, under, women_flag)

dates = [get_date(s) for s in df.date]
scores = [get_score(s) for s in df.score]
h_names = [process_name(s) for s in df.team_h]  # about 3s
a_names = [process_name(s) for s in df.team_a]  # about 3s